# ANÁLISIS Y PROCESAMIENTO DE DATOS DE LA ECH

Comenzaremos análizando los datos tal cual nos fueron entregados (en archivos .sav), para luego realizar un procesamiento de los mismos. A continuación se muestran algunos de los registros al azar de la ECH.

In [6]:
import pandas as pd
import geopandas
from sklearn.utils.extmath import weighted_mode
import numpy as np
import warnings

import warnings
warnings.filterwarnings('ignore')

# Se definen los años a procesar
firstECHyear = 2017
lastECHyear = 2019

dfsOriginales = {}

print("\n")

print("Mostramos algunos registros de la ECH, tal y como nos fueron entregados:\n")

for year in range(firstECHyear,lastECHyear+1):
    path = "ECH/{year}/H_{year}_Terceros.sav".format(year=year)
    df = pd.read_spss(path)

    dfsOriginales[str(year)] = df
    df.to_csv("ECH/{year}/ECH_{year}.csv".format(year=year), index=False)

    print('ECH ' + str(year) + ': \n')
    print('Cantidad de registros: ' + str(len(df)))
    print('Cantidad de columnas: ' + str(len(df.columns)) + '\n')

print('\n Mostramos 5 registros al azar de la ultima ECH: \n')
df.sample(5)





Mostramos algunos registros de la ECH, tal y como nos fueron entregados:

ECH 2017: 

Cantidad de registros: 45360
Cantidad de columnas: 166

ECH 2018: 

Cantidad de registros: 42282
Cantidad de columnas: 164

ECH 2019: 

Cantidad de registros: 42507
Cantidad de columnas: 164


 Mostramos 5 registros al azar de la ultima ECH: 



,numero,anio,mes,dpto,nomdpto,...,YSVL,lp_06,li_06,pobre06,indigente06
33777,2019046178,2019,Enero,Maldonado,MALDONADO,...,79621.0,5889.000000,2806.0,No pobre,No indigente
2111,2019002905,2019,Enero,Rocha,ROCHA,...,93892.0,10979.814773,2806.0,No pobre,No indigente
35260,2019048160,2019,Marzo,Montevideo,MONTEVIDEO,...,72689.0,14001.000000,3496.0,No pobre,No indigente
26884,2019036763,2019,Abril,Salto,SALTO,...,36801.0,37546.000078,3233.0,No pobre,No indigente
10695,2019014625,2019,Enero,Colonia,COLONIA,...,54423.0,22821.507586,3107.0,No pobre,No indigente


A primera vista podemos ver que hay muchísimas variables (160+ columnas) en estos datasets, y que la mayoría de ellas no son de nuestro interés. También notamos que entre los diferentes años no se tienen la misma cantidad de registros por encuesta (lo que tiene sentido, ya que no se tiene porque hacer exáctamente la misma cantidad de encuestas por año) ni la misma cantidad de columnas (o sea, cantidad de variables). A su vez también podemos notar que no todos los registros nos interesan, ya que como estámos trabajando en Montevideo, todo registro con el atributo 'dpto' distinto de 'Montevideo' no nos sirve. Por lo tanto, lo primero que haremos será filtrar los datos para quedarnos con los que nos interesan, y luego eliminar las columnas que no nos sirven.

In [7]:
dfsModificados = {}

print("\n")

print("Mostramos algunos de los registros luego de la selección de atributos y registros:\n")

for (year, df) in dfsOriginales.items():
    
    df.columns = map(str.lower, df.columns)
    dfModificado = df[['anio', 'secc', 'segm', 'dpto', 'pesoano', 'estred13', 'c1', 'd9', 'd20', 'd21_1', 'd21_5_1', 'ht11', 'd260', 'd21_14_1', 'c2', 'c3', 'c4']]
    dfModificado['dpto'] = dfModificado['dpto'].str.lower()
    dfModificado = dfModificado[dfModificado['dpto'] == 'montevideo']
    dfModificado = dfModificado.drop(columns=['dpto'])
    dfModificado['secc'] = dfModificado['secc'].astype(float)
    dfModificado['segm'] = dfModificado['segm'].astype(float)
    
    # renombre de variables
    dfModificado.rename(
        columns={'anio': 'año'}, inplace=True)
    dfModificado.rename(
        columns={'secc': 'seccion'}, inplace=True)
    dfModificado.rename(
        columns={'segm': 'segmento'}, inplace=True)
    dfModificado.rename(
        columns={'pesoano': 'peso_año'}, inplace=True)
    dfModificado.rename(
        columns={'ht11': 'ingresos_hogar'}, inplace=True)
    dfModificado.rename(
        columns={'estred13': 'nivel_economico_hogar'}, inplace=True)
    dfModificado.rename(
        columns={'c1': 'tipo_vivienda'}, inplace=True)
    dfModificado.rename(
        columns={'d9': 'cant_habitaciones'}, inplace=True)
    dfModificado.rename(
        columns={'d20': 'energia_cocinar'}, inplace=True)
    dfModificado.rename(
        columns={'d21_1': 'tiene_calefon'}, inplace=True)
    dfModificado.rename(
        columns={'d21_5_1': 'cant_televisores'}, inplace=True)
    dfModificado.rename(
        columns={'d260': 'energia_calefaccionar'}, inplace=True)
    dfModificado.rename(
        columns={'d21_14_1': 'cant_ac'}, inplace=True)
    dfModificado.rename(
        columns={'c2': 'material_pared'}, inplace=True)
    dfModificado.rename(
        columns={'c3': 'material_techo'}, inplace=True)
    dfModificado.rename(
        columns={'c4': 'material_piso'}, inplace=True)

    dfsModificados[year] = dfModificado
    
    print('ECH ' + str(year) + ': \n')
    print('Cantidad de registros luego del filtrado: ' + str(len(dfModificado)))
    print('Cantidad de columnas luego del filtrado: ' + str(len(dfModificado.columns)) + '\n')

print('\n Mostramos 5 registros al azar de la ultima ECH: \n')
dfModificado.sample(5)





Mostramos algunos de los registros luego de la selección de atributos y registros:

ECH 2017: 

Cantidad de registros luego del filtrado: 17815
Cantidad de columnas luego del filtrado: 16

ECH 2018: 

Cantidad de registros luego del filtrado: 16266
Cantidad de columnas luego del filtrado: 16

ECH 2019: 

Cantidad de registros luego del filtrado: 15628
Cantidad de columnas luego del filtrado: 16


 Mostramos 5 registros al azar de la ultima ECH: 



,año,seccion,segmento,peso_año,nivel_economico_hogar,...,energia_calefaccionar,cant_ac,material_pared,material_techo,material_piso
22101,2019,17.0,129.0,45.0,Montevideo Bajo,...,Energía eléctrica,0.0,"Ladrillos, ticholos o bloques con terminaciones",Liviano con cielo raso,"Cerámica, parqué, baldosas, moqueta o linóleo"
25430,2019,21.0,204.0,17.0,Montevideo Medio,...,Supergás,0.0,"Ladrillos, ticholos o bloques con terminaciones",Planchada de hormigón con protección (tejas u ...,"Cerámica, parqué, baldosas, moqueta o linóleo"
25090,2019,21.0,19.0,39.0,Montevideo Medio Bajo,...,Ninguna,0.0,"Ladrillos, ticholos o bloques con terminaciones",Planchada de hormigón con protección (tejas u ...,Alisado de hormigón
38340,2019,14.0,208.0,32.0,Montevideo Alto,...,Energía eléctrica,1.0,"Ladrillos, ticholos o bloques con terminaciones",Planchada de hormigón con protección (tejas u ...,"Cerámica, parqué, baldosas, moqueta o linóleo"
39305,2019,18.0,129.0,37.0,Montevideo Alto,...,Supergás,0.0,"Ladrillos, ticholos o bloques con terminaciones",Planchada de hormigón con protección (tejas u ...,"Cerámica, parqué, baldosas, moqueta o linóleo"


Ahora únicamente contamos con registros de Montevideo, por lo que podemos ver que la cantidad de registros de disminuyo significantemente (entre 15000 - 18000 por año). A su vez, pasamos de tener mas de 160 columnas a tener únicamente 16, lo que nos va a permitir trabajar con los datos de una manera mucho más sencilla. Como los nombres de las variables no eran muy amigables, los renombramos para que sean más fáciles de entender. 

Paso siguiente, debemos combinar los valores de los atributos de todos los registros que caigan en la misma seccion-segmento en un mismo año. De esta forma obtenemos un único registro representativo de toda la región por año. Pero nos dimos cuenta que no todas las secciones-segmentos fueron encuestadas en todos los años (o sea, que por lo menos se hizo uno encuesta dentro de la sección-segmento). Por lo tanto, deberemos quedarnos únicamente con las secciones-segmentos que fueron encuestadas en todos los años, ya que si no contamos con ninguna encuesta en alguna seccion-segmento no podremos obtener su promedio.

In [8]:
# leemos el archivo .shp que contiene los poligonos de las secciones y segmentos
dfSecSegm = geopandas.read_file("INE - IDE\censo11-segm\seg_hog11.shp")

# pasamos al crs adecuado (EPSG:4326 es estandar para latitud y longitud)
dfSecSegm .to_crs('EPSG:4326', inplace=True)

# filtrar unicamente para Montevideo (contiene secciones-segmentos de todo el país)
dfSecSegm  = dfSecSegm .loc[dfSecSegm ['nombdepto'] == 'MONTEVIDEO']

# nos quedamos únicamente con las que nos interesan
dfSecSegm = dfSecSegm[['seccion', 'segmento', 'geometry']]

print("\n")

print("Mostramos las secciones-segmentos que fueron encuestadas todos los años entre " + str(firstECHyear) + '-' + str(lastECHyear) + ":\n")

segmentosEncuestados = {}

for (year, df) in dfsModificados.items():
    segmentosEncuestados[str(year)] = df[['seccion', 'segmento']].drop_duplicates()

segmentosEncuestadosTodosLosAños = segmentosEncuestados[str(firstECHyear)]

for (year, df) in segmentosEncuestados.items():
    segmentosEncuestadosTodosLosAños = segmentosEncuestadosTodosLosAños.merge(df, on=['segmento','seccion'], how='inner')

print('Cantidad de secciones-segmentos totales en Montevideo = ' + str(len(dfSecSegm)))
print('Cantidad de secciones-segmentos encuestados todos los años entre ' + str(firstECHyear) + '-' + str(lastECHyear) + ' = ' + str(len(segmentosEncuestadosTodosLosAños)) + '\n')

# se alamacena en un csv
segmentosEncuestadosTodosLosAños.to_csv("ECH/SEGMENTOS_TODOS_AÑOS.csv".format(first=firstECHyear, last=lastECHyear), index=False)

# nos quedamos con las secciones-segmentos que se encuestan todos los años
for (year, df) in dfsModificados.items():
    df = df.merge(segmentosEncuestadosTodosLosAños, on=['segmento','seccion'], how='inner')
    dfsModificados[year] = df
    
# join dfSecSegm con segmentosEncuestadosTodosLosAños
dfSecSegm = dfSecSegm.merge(segmentosEncuestadosTodosLosAños, on=['segmento','seccion'], how='inner')
dfSecSegm.explore()





Mostramos las secciones-segmentos que fueron encuestadas todos los años entre 2017-2019:

Cantidad de secciones-segmentos totales en Montevideo = 1063
Cantidad de secciones-segmentos encuestados todos los años entre 2017-2019 = 854



De un total de 1063 secciones-segmentos que existen en Montevideo, 854 fueron encuestadas todos los años entre 2017-2019. Por lo tanto, nos quedamos con estas 854 secciones-segmentos para realizar el análisis. Procedemos ahora a combinar los valores de los atributos de todos los registros que caigan en la misma seccion-segmento en un mismo año, para estas 854 regiones que acabamos de filtrar. De esta forma obtenemos un único registro representativo de toda la región por año.

A su vez, se trata con el problema de la diferencia de formatos en las categorias de distintos años de la encuesta, unificandolo en un único formato. Como ejemplo está el caso de "material_pared", donde algunos años muestran la categoria "Ladrillos, ticholos o bloques con terminaciones" mientras que otros muestran "Ladrillos, ticholos o bloques, con terminaciones" (diferenciandose únicamente en una coma) que se consideran como la misma categoría. Ademas se ejecuta una transformación sobre algunas variables categóricas agrupando categorias que permitan hacer un conteo más facilmente, por ejemplo el caso de la variable tipo_vivienda donde agrupamos en dos categorias "casa" y "apartamento" registrándolas como 0 y 1 respectivamente. 

In [9]:
print("\n")

print("Pasamos todas las variables categoricas a numéricas y promediamos todos los atributos de las encuestas que caen en una misma seccion-segmento por año:\n")

def aggregateECH(df, args):

    filaResumen = pd.Series(dtype=object)

    # se registra el año de la entrada
    filaResumen['año'] = args

    # se suma la variable peso año
    filaResumen['peso_año'] = df['peso_año'].sum()

    # nivel_economico_hogar_prom: se calcula agrupando por seccion-segmento el valor ponderado.
    # 'Montevideo Bajo': 0, 'Montevideo Medio Bajo': 0.25, 'Montevideo Medio': 0.5, 'Montevideo Medio Alto': 0.75, 'Montevideo Alto': 1
    filaResumen['nivel_economico_hogar_prom'] = np.average(df['nivel_economico_hogar'], weights=df['peso_año'])

    # porcentaje_casas: en este caso, se mapea viviendas a casa o apartamento para tener un porcentaje definido.
    # 1: casa, 0: Apartamento
    filaResumen['porcentaje_casas'] = np.average(df['tipo_vivienda'], weights=df['peso_año'])

    # porcentaje_pared_ladrillo: Se calcula agrupando por seccion-segmento el porcentaje ponderado 
    # de todas las entradas que en la variable material_pared tienen la categorias ladrillo
    filaResumen['porcentaje_pared_ladrillo'] = np.average(df['material_pared'], weights=df['peso_año'])

    # porcentaje_techo_hormigon: Se calcula agrupando por seccion-segmento el porcentaje ponderado 
    # de todas las entradas que en la variable material_techo tienen la categorias hormigon
    filaResumen['porcentaje_techo_hormigon'] = np.average(df['material_techo'], weights=df['peso_año'])

    # porcentaje_techo_hormigon: Se calcula agrupando por seccion-segmento el porcentaje ponderado 
    # de todas las entradas que en la variable material_techo tienen la categorias hormigon    
    filaResumen['porcentaje_piso_con_terminacion'] = np.average(df['material_piso'], weights=df['peso_año'])

    # cant_habitaciones_prom: Se calcula, agrupando por seccion-segmento, el promedio ponderado de la variable cant_habitaciones
    filaResumen['cant_habitaciones_prom'] = np.average(df['cant_habitaciones'], weights=df['peso_año'])

    # porcentaje_cocina_electrica: Se calcula agrupando por seccion-segmento el porcentaje de ponderado 
    # de todas las entradas que en la variable energia_cocinar tienen la categorias hormigon
    filaResumen['porcentaje_cocina_electrica'] = round(df.where(df['energia_cocinar'] == "Energía eléctrica")['peso_año'].sum() / filaResumen['peso_año'], 2)

    # tiene_calefon_prom: Se calcula, agrupando por seccion-segmento, el promedio ponderado de todas las
    # entradas de la variable tiene_calefon 
    filaResumen['tiene_calefon_prom'] = np.average(df['tiene_calefon'].astype('float'), weights=df['peso_año'])

    # cant_televisores_prom: Se calcula agrupando por seccion-segmento el promedio ponderado de todas las
    # entradas para la variable cant_televisores
    filaResumen['cant_televisores_prom'] = np.average(df['cant_televisores'], weights=df['peso_año'])

    # ingresos_hogar_prom: Se calcula agrupando por seccion-segmento el promedio ponderado de todas las
    # entradas para la variable ingresos_hogar
    filaResumen['ingresos_hogar_prom'] = np.average(df['ingresos_hogar'], weights=df['peso_año'])

    # cantidad_ac_prom: Se calcula agrupando por seccion-segmento el promedio ponderado de todas las
    # entradas para la variable cantidad_ac
    if (args == 2013):
        filaResumen['cant_ac_prom'] = np.NaN
    else:
        filaResumen['cant_ac_prom'] = np.average(df['cant_ac'], weights=df['peso_año'])

    # porcentaje_energia_calefaccioanr: Se calcula agrupando por seccion-segmento el porcentaje ponderado 
    # de todas las entradas que en la variable energia_calefaccionar tienen la categoria electrica
    if (args == 2013):
        filaResumen['energia_calefaccionar_electrica'] = np.NaN
    else:
        filaResumen['porcentaje_calefaccio_electrica'] = round(df.where(df['energia_calefaccionar'] == "Energía eléctrica")['peso_año'].sum() / filaResumen['peso_año'], 2)
    
    return filaResumen

for (year, df) in dfsModificados.items():

    df.replace(['Sí', 'No'], [1, 0], inplace=True)
    df.replace({
    'tipo_vivienda': {
        'Apartamento o casa en complejo habitacional': 0,
        'Apartamento en edificio de altura': 0,
        'Apartamento en edificio de una planta': 1,
        'Local no construido para vivienda': 1,
        'Casa': 1, }
    , 'material_pared': {
        'Ladrillos, ticholos o bloques con terminaciones': 1,
        'Ladrillos, ticholos o bloques, con terminaciones': 1,
        'Ladrillos, ticholos o bloques sin terminaciones': 1,
        'Ladrillos, ticholos o bloques, sin terminaciones': 1,
        'Materiales livianos con revestimiento': 0,
        'Materiales livianos sin revestimiento': 0,
        'Adobe': 0,
        'Materiales de desecho': 0, }
    , 'material_techo': {
        'Planchada de hormigón con protección (tejas u otros)': 1,
        'Planchada de hormigón sin protección': 1,
        'Liviano con cielo raso': 0,
        'Liviano sin cielo raso': 0,
        'Quincha': 0,
        'Materiales de desecho': 0, }
    , 'material_piso': {
        'Cerámica, parqué, baldosas, moqueta o linóleo': 1,
        'Cerámica, parqué, moqueta, linóleo': 1,
        'Solo contrapiso sin piso': 0,
        'Alisado de hormigón': 0,
        'Tierra sin piso ni contrapiso': 0, }
    , 'nivel_economico_hogar': {
        'Montevideo Bajo': 0,
        'Montevideo Medio Bajo': 0.25,
        'Montevideo Medio': 0.5,
        'Montevideo Medio Alto': 0.75,
        'Montevideo Alto': 1, }
    }, inplace=True)

# ahora se aplica la funcion de transformacion a cada uno de los dataframes
ECH_promedio = pd.DataFrame()

for (year, df) in dfsModificados.items():
    ECH_promedio_year = df.groupby(['seccion', 'segmento']).apply(aggregateECH, args=year)
    # eliminamos peso_año porque ya no nos sirve
    ECH_promedio_year.drop('peso_año', axis=1, inplace=True)
    ECH_promedio_year.reset_index(inplace=True)

    print('ECH promediada ' + str(year) + ': \n')
    print('Cantidad de registros: ' + str(len(ECH_promedio_year)))
    print('Cantidad de columnas: ' + str(len(ECH_promedio_year.columns)) + '\n')

    ECH_promedio = pd.concat([ECH_promedio, ECH_promedio_year])

ECH_promedio.set_index(['seccion', 'segmento', 'año'], inplace=True)
ECH_promedio.to_csv("ECH/ECH_promedio.csv")

# show 10 columns max
pd.set_option('display.max_columns', 8)
print('\n Mostramos 5 registros al azar del dataset final de la ECH: \n')
ECH_promedio.sample(5)



Pasamos todas las variables categoricas a numéricas y promediamos todos los atributos de las encuestas que caen en una misma seccion-segmento por año:

ECH promediada 2017: 

Cantidad de registros: 854
Cantidad de columnas: 15

ECH promediada 2018: 

Cantidad de registros: 854
Cantidad de columnas: 15

ECH promediada 2019: 

Cantidad de registros: 854
Cantidad de columnas: 15


 Mostramos 5 registros al azar del dataset final de la ECH: 



,,,nivel_economico_hogar_prom,porcentaje_casas,porcentaje_pared_ladrillo,porcentaje_techo_hormigon,...,cant_televisores_prom,ingresos_hogar_prom,cant_ac_prom,porcentaje_calefaccio_electrica
seccion,segmento,año,,,,,,,,,
99.0,71.0,2019,0.25,1.000000,1.000000,0.434659,...,1.562500,60238.698864,0.480114,0.31
16.0,225.0,2018,0.25,1.000000,0.955267,0.367965,...,0.800866,63036.096922,0.282828,0.23
24.0,201.0,2019,1.00,0.120270,1.000000,1.000000,...,1.987838,161984.962162,1.208108,0.39
15.0,205.0,2018,0.50,0.000000,1.000000,1.000000,...,1.122754,55392.099800,0.164671,0.21
17.0,104.0,2017,0.25,0.803729,1.000000,0.522080,...,1.041217,55180.200360,0.251227,0.46


Como podemos observar, ahora el dataset de cada año tiene 854 registros, uno por cada seccion-segmento y 15 columnas ya que eliminamos 'peso_año' una vez que ya quedaron promediados los valores de los atributos. A su vez, todos los valores de las columnas son numéricos, lo que es necesario para los modelos de aprendizaje automático. Se unifican todos los años es un único dataset 'ECH_promedio' que tiene como índice los atributos 'seccion', 'segmento' y 'año'. Aquí termina el análisis de los datos de la ECH.